    Tiêu đề : Bài tập Project cá nhân
    Họ và tên : Nguyễn Văn Hiếu
    MSSV : 1412166
    Phiên bản : Python 2.7

## Nhận dạng giới tính
I. Phát biểu bài toán
    - Nhận dạng giới tính của một ảnh khuôn mặt mới trên cơ sở ảnh đã học hay huấn luyện thử nghiệm trên cơ sở dữ liệu ảnh FERET 1996
    - Tập huấn luyện ảnh gallery 
    - Tập ảnh kiểm tra : DUP1, DUP2, FB, FC
    - Sử dụng phương pháp PCA để rút trích đặc trưng sau đó sử dụng SVM để phân loại ảnh
    - Các kernel sử dụng :
        - Linear :
$$ <x,x'> = x^Tx' $$
        - Polynomial:
$$ <x, x'> = (x^Tx' + 1)^d $$
        - Radial Basic:
$$ <x, x'> = exp(-\frac{|x - x'|^2}{2\sigma^2}) $$
        - Sigmoid: 
$$ <x, x'> = tanh(ax^Tx' + b) $$

II. Kết quả
- Với n_components = 30      Thời gian chạy 19s
    
| Tập test      | RBF           | Polynomial  |Sigmoid   |Linear      |
|:------------- |:-------------:|:-----------:|:--------:|:----------:|
| DUP1          | 89.8892 %     |86.1496 %    | 80.7479 %|88.7812 %   |
| DUP2          | 90.1709  %    |82.4786 %    |80.3419 % |91.4530 %   |
| FB            | 90.7113 %     |80.5858 %    |68.2845 % |84.9372 %   |
| FC            | 78.8660 %     |71.1340 %    |63.9175 % |80.4124 %   |

- Với n_components = 50     Thời gian chạy 20s
    
| Tập test      | RBF           | Polynomial  |Sigmoid   |Linear      |
|:------------- |:-------------:|:-----------:|:--------:|:----------:|
| DUP1          | 90.5817 %     |84.2105 %    | 80.7479 %|89.3352 %   |
| DUP2          | 90.5983  %    |84.1880 %    |80.3419 % |90.1709 %   |
| FB            | 92.1339 %     |78.1590 %    |68.2845 % |86.0251 %   |
| FC            | 86.5979 %     |65.9794 %    |63.9175 % |80.9278 %   |

- Nhận xét:
    - Khi số chiều tăng lên thì thời gian chạy sẽ tăng lên và độ chính xác tăng lên
    - Ta thấy với một số chiều khác nhau và kernel khác nhau sẽ cho ra kết quả dự đoán khác nhau.
    - Kết quả sẽ tăng khi tăng số chiều lên nhưng đến một giới hạn sẽ bắt đầu giảm.

III. Lưu ý khi chạy
    - Cách tổ chức chương trình
    
```
1412166
│     
│
└───code
│   │   Bai1.py
│   │   Bai1.ipynb
│   │   Bai2.py
│   │   Bai2.ipynb
│   
│───test
│   │───dup1
│   │     │ xxxxxx.jpg
│   │     │ target.txt
│   │───dup2
│   │     │ xxxxxx.jpg
│   │     │ target.txt
│   │───fb
│   │     │ xxxxxx.jpg
│   │     │ target.txt
│   │───fc
│   │     │ xxxxxx.jpg
│   │     │ target.txt
│
│───train
│   │ xxxx.jpg
│   │ target.txt
```
        - Trong mỗi tập test và tập train sẽ có 1 file tên là target.txt đẻ gán nhãn cho đối tượng. 0 là nữ , 1 là nam

        - Tập train và test phải là tập các ảnh grayscale định dạng jpg

In [1]:
import datetime
import glob
import os
import warnings
import cv2
import numpy as np
from sklearn import svm
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore")


class Svm:
    def __init__(self, datasets, sexs, kernel):
        self.datasets = datasets
        self.sexs = sexs
        self.svm_obj = svm.SVC(kernel=kernel, C=1.0)
        self.svm_obj.fit(datasets, sexs)

    def classify(self, examples):
        sexs_predict = self.svm_obj.predict(examples)
        return sexs_predict


class Data:
    def __init__(self, directory):
        self.directory = directory
        self.datasets = []
        self.sexs = []

    def load(self):
        os.chdir(self.directory)
        for image in glob.glob("*.jpg"):
            pixel = cv2.imread(image, cv2.CV_LOAD_IMAGE_GRAYSCALE)
            self.datasets.insert(self.datasets.__len__(), pixel.flatten())
        file_target = 'target.txt'
        self.sexs = np.loadtxt(fname=file_target, dtype=int)
        os.chdir('../..')


class Recognition:
    def __init__(self, component):
        self.pca = PCA(n_components=component, whiten=True)
        self.training = []
        self.sexs = []

    def train(self, datasets, sexs):
        self.pca.fit(datasets)
        self.training = self.pca.transform(datasets)
        self.sexs = sexs

    def predict(self, examples, sexs, kernel):
        results = self.pca.transform(examples)
        svm = Svm(self.training, self.sexs, kernel)
        sexs_predict = svm.classify(results)
        return self.caculate_accuracy(sexs, sexs_predict)

    def compare(self, sex, sex_predict):
        if sex == sex_predict:
            return 1
        return 0

    def caculate_accuracy(self, sexs, sexs_predict):
        correct = np.sum(np.array([self.compare(sexs[i], sexs_predict[i]) for i in range(len(sexs))]))
        accuracy = float(correct) / len(sexs)
        return accuracy


def test(folder):
    dir = 'test/' + folder
    test_datasets = Data(dir)
    test_datasets.load()
    print("============%s=================" % (folder.upper()))
    accuracy_1 = reg.predict(test_datasets.datasets, test_datasets.sexs, 'rbf')
    print("Kernel RBF \t Accuracy : %f " %(accuracy_1))
    accuracy_2 = reg.predict(test_datasets.datasets, test_datasets.sexs, 'poly')
    print("Kernel Poly \t Accuracy : %f " % (accuracy_2))
    accuracy_3 = reg.predict(test_datasets.datasets, test_datasets.sexs, 'sigmoid')
    print("Kernel Sigmoid \t Accuracy : %f " % (accuracy_3))
    accuracy_4 = reg.predict(test_datasets.datasets, test_datasets.sexs, 'linear')
    print("Kernel Linear \t Accuracy : %f " % (accuracy_4))

os.chdir('..')
train_datasets = Data('train/gallery')
train_datasets.load()
reg = Recognition(30)
reg.train(train_datasets.datasets, train_datasets.sexs)

In [2]:
test('dup1')

============DUP1=================
Kernel RBF 	 Accuracy : 0.898892 
Kernel Poly 	 Accuracy : 0.861496 
Kernel Sigmoid 	 Accuracy : 0.807479 
Kernel Linear 	 Accuracy : 0.887812 


In [3]:
test('dup2')

============DUP2=================
Kernel RBF 	 Accuracy : 0.901709 
Kernel Poly 	 Accuracy : 0.824786 
Kernel Sigmoid 	 Accuracy : 0.803419 
Kernel Linear 	 Accuracy : 0.914530 


In [4]:
test('fb')

============FB=================
Kernel RBF 	 Accuracy : 0.907113 
Kernel Poly 	 Accuracy : 0.805858 
Kernel Sigmoid 	 Accuracy : 0.682845 
Kernel Linear 	 Accuracy : 0.849372 


In [5]:
test('fc')

============FC=================
Kernel RBF 	 Accuracy : 0.788660 
Kernel Poly 	 Accuracy : 0.711340 
Kernel Sigmoid 	 Accuracy : 0.639175 
Kernel Linear 	 Accuracy : 0.804124 
